In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import random
from sklearn.metrics import accuracy_score, log_loss
from scipy import optimize
from keras import losses
from keras import backend as K
import tensorflow as tf
from sklearn import neural_network

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier
import keras
from keras import losses
from functools import partial
import math

%matplotlib notebook

# Preprocessing

In [5]:
data = pd.read_csv('leaf.csv', header=None)
data.drop([1],inplace=True,axis=1)
X = data.values[:,1:]
Y_first = data.values[:,0]
Y = []
out = np.zeros(shape=[36])
for i in range(np.shape(Y_first)[0]):
    out[int(Y_first[i])-1]=1
    Y.append(out)
    out = np.zeros(shape=[36])
Y = np.array(Y)
# min_max_scaler = preprocessing.MinMaxScaler()
# X_scaled = min_max_scaler.fit_transform(X)
# df_normalized = pd.DataFrame(X_scaled)

r,c = np.shape(X)

total_samples = list(np.arange(0,r,1))
test_samples = random.sample(total_samples,int(np.round(len(total_samples)*0.2)))
train_samples = list(set(total_samples) - set(test_samples))

# Splitting Data

In [6]:
trainX = X[train_samples]
trainX_scaled = (trainX-np.min(trainX))/(np.max(trainX)-np.min(trainX))
trainY = Y[train_samples]
trainY_comp = Y_first[train_samples]

testX = X[test_samples]
testX_scaled = (testX-np.min(testX))/(np.max(testX)-np.min(testX))
testY = Y[test_samples]
testY_comp = Y_first[test_samples]
print(len(trainX[0]))
print(len(trainY[0]))

14
36


In [7]:
def logloss(true_label, predicted, eps=1e-15):
  p = np.clip(predicted, eps, 1 - eps)
  if true_label == 1:
    return -np.log(p)
  else:
    return 0

In [13]:
def sa(energyfunc, initials, epochs, tempfunc, iterfunc, proposalfunc):
    accumulator=[]
    best_solution = old_solution = initials['solution']
    T=initials['T']
    length=initials['length']
    best_energy = old_energy = energyfunc(old_solution)
    accepted=0
    total=0
    for index in range(epochs):
        print("Epoch", index)
        if index > 0:
            T = tempfunc(T)
            length=iterfunc(length)
        print("Temperature", T, "Length", length)
        for it in range(length):
            total+=1
            new_solution = proposalfunc(old_solution)
            new_energy = energyfunc(new_solution)
            # Use a min here as you could get a "probability" > 1
            alpha = min(1, np.exp((old_energy - new_energy)/T))
            if ((new_energy < old_energy) or (np.random.uniform() < alpha)):
                # Accept proposed solution
                accepted+=1
                accumulator.append((T, new_solution, new_energy))
                if new_energy < best_energy:
                    # Replace previous best with this one
                    best_energy = new_energy
                    best_solution = new_solution
                    best_index=total
                    best_temp=T
                old_energy = new_energy
                old_solution = new_solution
            else:
                # Keep the old stuff
                accumulator.append((T, old_solution, old_energy))
    
    best_meta=dict(index=best_index, temp=best_temp)
    print("frac accepted", accepted/total, "total iterations", total, 'bmeta', best_meta)
    return best_meta, best_solution, best_energy, accumulator

#  Neural Network

In [43]:
def nn_objective(x_u):
    num_inputs = 14
    num_outputs = 36

    hidden_layer_neurons = 15
    x = x_u[0]
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = x[:weights_all]
    bias_all = x[weights_all:]
    iw = nn_weights[0:iw_size]
    ow = nn_weights[iw_size:]
    b1 = bias_all[:hidden_layer_neurons]
    b2 = bias_all[hidden_layer_neurons:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))
#     l1 = 1/(1+np.exp(-(np.dot(trainX,w1)+b1)))
    l1 = np.maximum(np.dot(trainX, w1)+b1, 0)
    x_i = np.dot(l1, w2)+b2
    l2 = np.array(np.exp(x_i)/ np.exp(x_i).sum(axis=1)[:,None])  
#     l2 = np.exp(x_i) / (1 + np.exp(x_i))
    l2_act = np.argmax(l2,axis=1)+1
#     predict_out = l2_act.flatten()
#     acc = 1-accuracy_score(trainY_comp,predict_out)
#     er_t = []
#     er_i = 0
#     for i in range(np.shape(trainY)[0]):
#         for j in range(np.shape(trainY)[1]):
#             er_i += logloss(np.array(trainY[i,j]),np.array(l2[i,j]))
#         #er_t.append(er_i)
#     er = np.mean(er_i)
    output = np.clip(l2, 1e-15, 1. - 1e-15)
    er= -np.sum(trainY * np.log(l2))
    return er



In [57]:
num_inputs = 14
num_outputs = 36

hidden_layer_neurons = 15

weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
bias_all = hidden_layer_neurons+num_outputs

tw = weights_all+bias_all 
w0 = 2*np.random.random((1,tw))-2
b = [(-1,1)]
for i in range(len(w0[0])-1):
    b.append((-1,1))

tf = lambda t: 0.8*t #temperature function
itf = lambda length: math.ceil(1.2*length) #iteration function
pfxs = lambda s, w0: w0 + s*np.random.normal()
inits=dict(solution=w0, length=10, T=100)
pf = partial(pfxs, 0.1)

bmeta, bs, be, out = sa(nn_objective, inits, 100, tf, itf, pf)
print(be)      
# mut_all = np.array([0.0,0.25,0.5])
# crossp_all = np.array([0.4,0.6,0.8])
# popsize_all = np.array([50,100,200])
# r_all = []
# for i in range(len(mut_all)):
#     print(i)
#     for j in range(len(crossp_all)):
#         for k in range(len(popsize_all)):
#             res1 = list(de(nn_objective,b,mut=mut_all[i], crossp=crossp_all[j], popsize=popsize_all[k], its=500))
#             r_all.append(np.array([mut_all[i],crossp_all[j],popsize_all[k],res1[1]]))
#             print(res1[1])
#res = optimize.basinhopping(nn_objective, w0[0], niter=1000,disp=True)
# res1 = list(de(nn_objective,b,mut=0.0, crossp=0.9, popsize=50, its=1000))


Epoch 0
Temperature 100 Length 10
Epoch 1
Temperature 80.0 Length 12
Epoch 2
Temperature 64.0 Length 15
Epoch 3
Temperature 51.2 Length 18
Epoch 4
Temperature 40.96000000000001 Length 22
Epoch 5
Temperature 32.76800000000001 Length 27
Epoch 6
Temperature 26.21440000000001 Length 33
Epoch 7
Temperature 20.97152000000001 Length 40
Epoch 8
Temperature 16.777216000000006 Length 48
Epoch 9
Temperature 13.421772800000006 Length 58
Epoch 10
Temperature 10.737418240000006 Length 70
Epoch 11
Temperature 8.589934592000004 Length 84
Epoch 12
Temperature 6.871947673600004 Length 101
Epoch 13
Temperature 5.497558138880003 Length 122
Epoch 14
Temperature 4.398046511104003 Length 147
Epoch 15
Temperature 3.5184372088832023 Length 177
Epoch 16
Temperature 2.814749767106562 Length 213
Epoch 17
Temperature 2.25179981368525 Length 256
Epoch 18
Temperature 1.8014398509482001 Length 308
Epoch 19
Temperature 1.4411518807585602 Length 370
Epoch 20
Temperature 1.1529215046068482 Length 444
Epoch 21
Temperatur

KeyboardInterrupt: 

In [30]:
print(np.shape(r_all))
all_vals = np.reshape(r_all,(27,4))
b_idx = np.argmin(all_vals[:,-1])
b_vals = all_vals[16,:]
print(b_vals)

(27, 4)
[  2.50000000e-01   8.00000000e-01   1.00000000e+02   5.03300281e+02]


In [35]:
res1_f = list(de(nn_objective,b,mut=b_vals[0], crossp=b_vals[1], popsize=int(b_vals[2]), its=10000))

In [39]:
x = res1_f[0]

def nn_objective_test(x):
    num_inputs = 14
    num_outputs = 36

    hidden_layer_neurons = 15
    
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = x[:weights_all]
    bias_all = x[weights_all:]
    iw = nn_weights[0:iw_size]
    ow = nn_weights[iw_size:]
    b1 = bias_all[:hidden_layer_neurons]
    b2 = bias_all[hidden_layer_neurons:]

    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))
    l1 = np.maximum(np.dot(testX, w1)+b1, 0)
    x_i = np.dot(l1, w2)+b2
    l2 = np.array(np.exp(x_i)/ np.exp(x_i).sum(axis=1)[:,None])  
    print(np.sum(l2,axis=1))
#     l2 = np.exp(x_i) / (1 + np.exp(x_i))
    l2_act = np.argmax(l2,axis=1)+1
    predict_out = l2_act
    predict_out = predict_out.flatten()
    acc = 1-accuracy_score(testY_comp,predict_out)
    print(testY_comp)
    print(predict_out)
    return acc

test_miss = nn_objective_test(x)
print(test_miss)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
[ 26.  35.  29.  13.  15.  11.  13.  14.  27.   7.   8.  35.  11.   1.   7.
   8.  10.  25.  34.  13.   2.   6.   8.   2.  13.   7.  24.  26.  12.  34.
  13.  36.  24.  26.   6.  12.   1.  22.  29.  27.   6.  27.   7.  24.  26.
   7.  22.  10.  13.  12.  29.  15.  14.  13.   1.  11.  26.  10.  30.   5.
  23.  36.  32.  11.  32.   9.   5.   6.]
[27 28 29 33 15 11 33 28 33  7  8 22 11  1  5  8 15 25 34 33  2  6  8 32 33
  5 27 30 35 34 33 36  4 24 11 12  1 22 29 27 23 27  7 24  4  5  5 30 33 12
 29 15 28 33  1 11 32  9 10  5 23 36 32 11 32  3  5 23]
0.485294117647


In [38]:
import h5py
file_name = "weights.best_leaf.hdf5"
data = h5py.File(file_name,'r')

for key in data.keys():
    print(key) #Names of the groups in HDF5 file.

# group = data['model_weights']
# data = group['model_weights']

d = list(data['model_weights'])
print(d[:])

model_weights
optimizer_weights
['dense_1', 'dense_2']


In [ ]:
from keras.models import load_model
model = load_model("weights.best_leaf.hdf5")
print(model.keys())

In [ ]:
import h5py
f = h5py.File("weights.best_leaf.hdf5", 'r')
print(list(f.keys()))
#will get a list of layer names which you can use as index
d = f['model_weights']['dense_1']
print(list(d.get('dense_1').keys()))
d.get('dense_1').get('bias:0')[()]

In [15]:
x = res1[0]

def nn_keras(x):
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = x[:weights_all]
    bias_all = x[weights_all:]
    iw = nn_weights[0:iw_size]
    ow = nn_weights[iw_size:]
    b1 = bias_all[:hidden_layer_neurons]
    b2 = bias_all[hidden_layer_neurons:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))

    w1_all = []
    w1_all.append(w1)
    w1_all.append(b1)

    w2_all = []
    w2_all.append(w2)
    w2_all.append(b2)

    model = Sequential()
    model.add(Dense(15, input_dim=14, kernel_initializer='normal', activation='relu'))
    model.add(Dense(36, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.layers[0].set_weights(w1_all)
    model.layers[1].set_weights(w2_all)
    out = model.predict(trainX)
    output = np.clip(out, 1e-15, 1. - 1e-15)
    er= -np.sum(trainY * np.log(output))
    predict_out = np.argmax(out,axis=1)+1
#     acc = 1-accuracy_score(testY_comp,predict_out)
#     print(acc)
    return er
    


NameError: name 'res1' is not defined